# Install All Necessary Packages

In [4]:
import pandas as pd
import requests
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
pip install BeautifulSoup4

Note: you may need to restart the kernel to use updated packages.


In [19]:
from bs4 import BeautifulSoup as bs

In [43]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

### Testing Out How the Selenium Library Works

In [5]:
#Create the Driver
driver = webdriver.Chrome(ChromeDriverManager().install())

page_url = "https://evehiclesdb.com/category/transport/electric-cars/"
driver.get(page_url)

C:\Users\kagom\AppData\Local\Temp\ipykernel_5416\1132155773.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [6]:
ev_categories = driver.find_elements(By.CLASS_NAME, "brand_list_item")

ev_categories[0].get_attribute('href')

In [15]:
ev_element = driver.find_elements(By.CLASS_NAME, "top_cars_list_item")

ev_element

# Full Code

## Generate a list of URLs to scrape from

In [67]:
# Create Driver
driver = webdriver.Chrome(ChromeDriverManager().install())

#Go to the EVs DB page
page_url = "https://evehiclesdb.com/category/transport/electric-cars/"
driver.get(page_url)

#Find EV makes
ev_make = driver.find_elements(By.CLASS_NAME, "brand_list_item")

ev_makes = [] #creates an empty list
for make in ev_make:
    make_url = make.get_attribute('href')
    make_name = make.text
    ev_makes.append({'ev make' : make_name, 'url': make_url})

C:\Users\kagom\AppData\Local\Temp\ipykernel_7180\3269401552.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [18]:
#Testing cell
ev_makes

[{'ev make': 'Aiways (1)',
  'url': 'https://evehiclesdb.com/brands/aiways?category=electric-cars'},
 {'ev make': 'Aston Martin (1)',
  'url': 'https://evehiclesdb.com/brands/aston-martin?category=electric-cars'},
 {'ev make': 'Audi (13)',
  'url': 'https://evehiclesdb.com/brands/audi?category=electric-cars'},
 {'ev make': 'BAIC (1)',
  'url': 'https://evehiclesdb.com/brands/baic?category=electric-cars'},
 {'ev make': 'BMW (9)',
  'url': 'https://evehiclesdb.com/brands/bmw?category=electric-cars'},
 {'ev make': 'Byton (3)',
  'url': 'https://evehiclesdb.com/brands/byton?category=electric-cars'},
 {'ev make': 'Citroen (8)',
  'url': 'https://evehiclesdb.com/brands/citroen?category=electric-cars'},
 {'ev make': 'CUPRA (4)',
  'url': 'https://evehiclesdb.com/brands/cupra?category=electric-cars'},
 {'ev make': 'Dacia (1)',
  'url': 'https://evehiclesdb.com/brands/dacia?category=electric-cars'},
 {'ev make': 'DS (1)',
  'url': 'https://evehiclesdb.com/brands/ds?category=electric-cars'},
 {'

In [68]:
ev_model = [] #list for all the EV models

for model in ev_makes:
    driver.get(model['url'])
    
    ev_models = driver.find_elements(By.CLASS_NAME, "top_cars_list_item")
    
       
    for models in ev_models:
        #models_url = 'https://evehiclesdb.com/transport/'+ model +'/'
        ev_model.append({'ev make': model['ev make'], 'model': models.text})

In [69]:
#This will generate the url for each EV
ev_df = pd.DataFrame(ev_model)
ev_df['url string'] = ev_df['model'].str.split(' ').str.join("-").str.lower()

ev_df['url'] = 'https://evehiclesdb.com/transport/'+ev_df['url string']+'/'
#some of the URL pages in this database have buttons and we need to always make sure we are accessing "Specifications"
#driver.find_element(By.XPATH, '//span[text()="Specifications"]').click()



NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//span[text()="Specifications"]"}
  (Session info: chrome=109.0.5414.74)
Stacktrace:
Backtrace:
	(No symbol) [0x01206643]
	(No symbol) [0x0119BE21]
	(No symbol) [0x0109DA9D]
	(No symbol) [0x010D1342]
	(No symbol) [0x010D147B]
	(No symbol) [0x01108DC2]
	(No symbol) [0x010EFDC4]
	(No symbol) [0x01106B09]
	(No symbol) [0x010EFB76]
	(No symbol) [0x010C49C1]
	(No symbol) [0x010C5E5D]
	GetHandleVerifier [0x0147A142+2497106]
	GetHandleVerifier [0x014A85D3+2686691]
	GetHandleVerifier [0x014ABB9C+2700460]
	GetHandleVerifier [0x012B3B10+635936]
	(No symbol) [0x011A4A1F]
	(No symbol) [0x011AA418]
	(No symbol) [0x011AA505]
	(No symbol) [0x011B508B]
	BaseThreadInitThunk [0x76227D69+25]
	RtlInitializeExceptionChain [0x77B5BB9B+107]
	RtlClearBits [0x77B5BB1F+191]


## Function to Scrape an HTML Table

In [45]:
def table_scrape(table):
    """Parses an html segment started with tag <table> followed 
    by multiple <tr> (table rows) and inner <td> (table data) tags.
    It returns a list of rows with inner columns.
    """
    def row_scrape(tr, coltag='td'): #td (data) 
        return [td.get_text(strip=True) for td in tr.find_all(coltag)]
    rows = []
    trs = table.find_all('tr')
    for tr in trs: #for every table row
        rows.append(row_scrape(tr, 'td')) #data row
    return rows

In [51]:
#Testing a single URL with the scraping function created in the cell before
url_test = 'https://evehiclesdb.com/transport/honda-e/'

response_test = requests.get(url_test)
soup_test = bs(response_test.text, 'html.parser')
ev_table_test = soup_test.find('table', dir='ltr')

test_table = pd.DataFrame(table_scrape(ev_table_test))
#test_table[:3]

In [49]:
test_table 

,0,1
0,Performance,
1,Acceleration 0 – 100 km/h,9.0 sec
2,Top Speed,145 km/h
3,Electric Range,170 km
4,Total Power,100 kW (136 PS)
...,...,...
78,Platform,No Data
79,Car Body,Hatchback
80,Segment,B – Small
81,Roof Rails,No


## Scrape ALL HTML Tables from ALL the URLs 

In [63]:
ev_list = list(ev_df['url']) #List of all the urls for each EV

#comp_ev_table = pd.DataFrame
evs = []
count = 0
# while count <= len(ev_list): #While count is less than or equal to the total number of EVs in the database
#     url = ev_list[count] #this is the individual url string for each EV
#     response = requests.get(url)
#     soup = bs(response.text, 'html.parser')
#     ev_table = soup.find('table', dir='ltr')
   
#     comp_ev_table = table_scrape(ev_table)

#     count += 1 

#url = ev_list[count] #this is the individual url string for each EV
response = requests.get(ev_list[0])
soup = bs(response.text, 'html.parser')
ev_table = soup.find('table', dir='ltr')
   
comp_ev = pd.DataFrame(table_scrape(ev_table))
comp_ev


,0,1
0,Performance,
1,Acceleration 0 – 100 km/h,7.5 sec
2,Top Speed,150 km/h
3,Electric Range,330 km
4,Total Power,150 kW (204 PS)
...,...,...
86,Platform,No Data
87,Car Body,SUV
88,Segment,JC – Medium
89,Roof Rails,Yes


In [89]:
ev_list[17]

'https://evehiclesdb.com/transport/bmw-ix-xdrive50/'

In [90]:
#perhaps need to write an if statement that essentially says, if the first cell of the first row says Performance, continue. If not, move on. 
#Otherwise, it'll keep throwing errors.


response = requests.get(ev_list[17])
soup = bs(response.text, 'html.parser')
ev_table = soup.find('table', dir='ltr')
   
comp_ev = pd.DataFrame(table_scrape(ev_table))
comp_ev

,0,1,2,3,4
0,Charging Point,Max. Power,Power,Time,Rate
1,Wall Plug (2.3 kW),230V / 1x10A,2.3 kW,54 hours,9 km/h
2,1-phase 16A (3.7 kW),230V / 1x16A,3.7 kW,33h30m,15 km/h
3,1-phase 32A (7.4 kW),230V / 1x32A,7.4 kW,16h45m,30 km/h
4,3-phase 16A (11 kW),400V / 3x16A,11 kW,11h15m,45 km/h
5,3-phase 32A (22 kW),400V / 3x16A,11 kW,11h15m,45 km/h


In [54]:
type(ev_list[1])

str